In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.manifold import TSNE
import gc
from optuna.integration.lightgbm import LightGBMTunerCV
from sklearn.model_selection import KFold

In [2]:
def imputation(df):
    for col in df:
        if(df[col].dtype=='object'):
            df[col].replace(np.nan,df[col].mode().iloc[0],inplace=True)
        else:
            df[col].replace(np.nan,df[col].mean(),inplace=True)
    print("Imputaion Done")
    return df

def label_encoder(df): #based on https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/?ref=rp
    le = LabelEncoder()
    le_count = 0
    for col in df: 
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str)
            #if len(list(df[col].unique())) <= 2:   #uncomment this line to encode columns with total unique values less than 3
            le.fit(df[col])
            df[col] = le.transform(df[col])
            le_count += 1
    print('%d columns were label encoded.' % le_count)

def Remove_anomaly(name,lst,typ):
    for var in lst:
        temp_data = name[var]
        count = 0
        count1 = 0
        for val in temp_data:
            if typ == 1:
                if val<0:
                    count +=1
                    name.at[count1,var] = np.nan
            else:
                if val>0:
                    count +=1
                    name.at[count1,var] = np.nan
            count1+=1
        print('Total anomalies ='+str(count))   
        #Plot_hist(temp_data,var)
        return

def clean_features(d_file,no_unique= 0,percentage= 100):
    cat_cols = []
    int_cols = []
    
    ln = len(d_file.columns)
    #no_unique, number of unique values
    #percentage, max percentage the no_unique can take
    
    for col in d_file:
        k = (d_file[col].value_counts()/d_file[col].count())*100
        if sum(list(k.head(no_unique)))<percentage:
            if d_file[col].dtypes == "object":
                cat_cols.append(col)
            else: int_cols.append(col)
                
    #removing all features execpt the ones that have passed the above test
    d_file = d_file.drop(train_file.columns.difference(cat_cols+int_cols),1)
    
    print(ln-len(d_file.columns),"Features dropped")
    

def treat_iqr(d_file):
    count = 0
    for col in d_file:
        if col != 'TARGET':
            skew_prev = d_file[col].skew()
    
            Q1 = d_file[col].quantile(0.25)
            Q2 = d_file[col].quantile(0.75)
            IQR = Q2-Q1
        
            Q3 = Q2-3*IQR
            Q4 = Q3+3*IQR
        
            d_file[col] = np.where(d_file[col]<Q3,Q3,d_file[col])
            d_file[col] = np.where(d_file[col]>Q4,Q4,d_file[col])
        
            skew_after = d_file[col].skew()
        
            if (skew_prev != skew_after):
                count += 1
            
            print(col,"Skew_prev:",skew_prev,"Skew_after:",skew_after)
    
    print("Outliers changed in:",count,"columns")

def reduce_memory(df):
    """Reduce memory usage of a dataframe by setting data types. """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Initial df memory usage is {:.2f} MB for {} columns'
          .format(start_mem, len(df.columns)))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                # Can use unsigned int here too
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    memory_reduction = 100 * (start_mem - end_mem) / start_mem
    print('Final memory usage is: {:.2f} MB - decreased by {:.1f}%'.format(end_mem, memory_reduction))
    return df

def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """Create a new column for each categorical value in categorical columns. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object' and "SK_ID" not in col]
    original_columns = [col for col in original_columns if col not in categorical_columns]
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns, original_columns

In [3]:
train_file = pd.read_csv('../input/iiitb2020-home-credit-default-risk/application_train.csv',sep = ',', low_memory = False)
print("Train_file Imported")
test_file = pd.read_csv('../input/iiitb2020-home-credit-default-risk/application_test.csv',sep = ',', low_memory = False)
print("Test_file Imported")
prev_app = pd.read_csv('../input/iiitb2020-home-credit-default-risk/previous_application.csv', sep = ',', low_memory = False)
print("Previous Application Imported")
bureau_bal = pd.read_csv('../input/iiitb2020-home-credit-default-risk/bureau_balance.csv',sep=',', low_memory = False)
print("Bureau Balance Imported")
pos_cash = pd.read_csv('../input/iiitb2020-home-credit-default-risk/POS_CASH_balance.csv',sep = ',', low_memory = False)
print("POS Cash Balance Imported")
bureau = pd.read_csv('../input/iiitb2020-home-credit-default-risk/bureau.csv', sep = ',', low_memory = False)
print("Bureau Imported")
credit_card_bal = pd.read_csv('../input/iiitb2020-home-credit-default-risk/credit_card_balance.csv', sep=',', low_memory = False)
print("Credit Card Balance Imported")
installment_pay = pd.read_csv('../input/iiitb2020-home-credit-default-risk/installments_payments.csv',sep=',', low_memory = False)
print("Installment Payments Imported")
print("Import Done")

Train_file Imported
Test_file Imported
Previous Application Imported
Bureau Balance Imported
POS Cash Balance Imported
Bureau Imported
Credit Card Balance Imported
Installment Payments Imported
Import Done


In [4]:
train_file.DAYS_EMPLOYED = train_file.DAYS_EMPLOYED.astype(float)
Remove_anomaly(train_file,["DAYS_EMPLOYED"],2)

cols = ["DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH"]
for i in cols:
    train_file[i] = train_file[i]/(-365.25)
    
# Credit ratios
train_file['CREDIT_TO_ANNUITY_RATIO'] = train_file['AMT_CREDIT'] / train_file['AMT_ANNUITY']
train_file['CREDIT_TO_GOODS_RATIO'] = train_file['AMT_CREDIT'] / train_file['AMT_GOODS_PRICE']

# Income ratios
train_file['ANNUITY_TO_INCOME_RATIO'] = train_file['AMT_ANNUITY'] / train_file['AMT_INCOME_TOTAL']
train_file['CREDIT_TO_INCOME_RATIO'] = train_file['AMT_CREDIT'] / train_file['AMT_INCOME_TOTAL']
train_file['INCOME_TO_EMPLOYED_RATIO'] = train_file['AMT_INCOME_TOTAL'] / train_file['DAYS_EMPLOYED']
train_file['INCOME_TO_BIRTH_RATIO'] = train_file['AMT_INCOME_TOTAL'] / train_file['DAYS_BIRTH']

# Time ratios
train_file['EMPLOYED_TO_BIRTH_RATIO'] = train_file['DAYS_EMPLOYED'] / train_file['DAYS_BIRTH']
train_file['ID_TO_BIRTH_RATIO'] = train_file['DAYS_ID_PUBLISH'] / train_file['DAYS_BIRTH']
train_file['CAR_TO_BIRTH_RATIO'] = train_file['OWN_CAR_AGE'] / train_file['DAYS_BIRTH']
train_file['CAR_TO_EMPLOYED_RATIO'] = train_file['OWN_CAR_AGE'] / train_file['DAYS_EMPLOYED']
train_file['PHONE_TO_BIRTH_RATIO'] = train_file['DAYS_LAST_PHONE_CHANGE'] / train_file['DAYS_BIRTH']

Total anomalies =35869


In [5]:
test_file.DAYS_EMPLOYED = test_file.DAYS_EMPLOYED.astype(float)
Remove_anomaly(test_file,["DAYS_EMPLOYED"],2)

cols = ["DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH"]
for i in cols:
    test_file[i] = test_file[i]/(-365.25)
    
# Credit ratios
test_file['CREDIT_TO_ANNUITY_RATIO'] = test_file['AMT_CREDIT'] / test_file['AMT_ANNUITY']
test_file['CREDIT_TO_GOODS_RATIO'] = test_file['AMT_CREDIT'] / test_file['AMT_GOODS_PRICE']

# Income ratios
test_file['ANNUITY_TO_INCOME_RATIO'] = test_file['AMT_ANNUITY'] / test_file['AMT_INCOME_TOTAL']
test_file['CREDIT_TO_INCOME_RATIO'] = test_file['AMT_CREDIT'] / test_file['AMT_INCOME_TOTAL']
test_file['INCOME_TO_EMPLOYED_RATIO'] = test_file['AMT_INCOME_TOTAL'] / test_file['DAYS_EMPLOYED']
test_file['INCOME_TO_BIRTH_RATIO'] = test_file['AMT_INCOME_TOTAL'] / test_file['DAYS_BIRTH']

# Time ratios
test_file['EMPLOYED_TO_BIRTH_RATIO'] = test_file['DAYS_EMPLOYED'] / test_file['DAYS_BIRTH']
test_file['ID_TO_BIRTH_RATIO'] = test_file['DAYS_ID_PUBLISH'] / test_file['DAYS_BIRTH']
test_file['CAR_TO_BIRTH_RATIO'] = test_file['OWN_CAR_AGE'] / test_file['DAYS_BIRTH']
test_file['CAR_TO_EMPLOYED_RATIO'] = test_file['OWN_CAR_AGE'] / test_file['DAYS_EMPLOYED']
test_file['PHONE_TO_BIRTH_RATIO'] = test_file['DAYS_LAST_PHONE_CHANGE'] / test_file['DAYS_BIRTH']

Total anomalies =19505


In [6]:
cols = ["DAYS_DECISION","DAYS_FIRST_DRAWING","DAYS_FIRST_DUE","DAYS_LAST_DUE_1ST_VERSION","DAYS_LAST_DUE","DAYS_TERMINATION"]
for i in cols:
    prev_app[i] = prev_app[i]/(-365.25)

prev_app['APPLICATION_CREDIT_DIFF'] = prev_app['AMT_APPLICATION'] - prev_app['AMT_CREDIT']
prev_app['APPLICATION_CREDIT_RATIO'] = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT']
prev_app['CREDIT_TO_ANNUITY_RATIO'] = prev_app['AMT_CREDIT'] / prev_app['AMT_ANNUITY']
prev_app['DOWN_PAYMENT_TO_CREDIT'] = prev_app['AMT_DOWN_PAYMENT'] / prev_app['AMT_CREDIT']

In [7]:
cols = ["MONTHS_BALANCE"]
for i in cols:
    pos_cash[i] = pos_cash[i]/(-365.25)

pos_cash['LATE_PAYMENT'] = pos_cash['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

In [8]:
cols = ["DAYS_CREDIT","DAYS_ENDDATE_FACT"]
for i in cols:
    bureau[i] = bureau[i]/(-365.25)
    
bureau['CREDIT_DURATION'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_ENDDATE_FACT'] - bureau['DAYS_CREDIT_ENDDATE']

bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']

In [9]:
cols = ["MONTHS_BALANCE"]
for i in cols:
    bureau_bal[i] = bureau_bal[i]/(-365.25)

In [10]:
cols = ["MONTHS_BALANCE"]
for i in cols:
    credit_card_bal[i] = credit_card_bal[i]/(-365.25)
    
list1 = ["AMT_BALANCE","AMT_DRAWINGS_ATM_CURRENT","AMT_DRAWINGS_CURRENT", "AMT_RECEIVABLE_PRINCIPAL", "AMT_RECIVABLE","AMT_TOTAL_RECEIVABLE"]
Remove_anomaly(credit_card_bal,list1,1)   

credit_card_bal['LIMIT_USE'] = credit_card_bal['AMT_BALANCE'] / credit_card_bal['AMT_CREDIT_LIMIT_ACTUAL']
credit_card_bal['PAYMENT_DIV_MIN'] = credit_card_bal['AMT_PAYMENT_CURRENT'] / credit_card_bal['AMT_INST_MIN_REGULARITY']
credit_card_bal['LATE_PAYMENT'] = credit_card_bal['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)


Total anomalies =2345


In [11]:
cols = ["DAYS_INSTALMENT","DAYS_ENTRY_PAYMENT"]
for i in cols:
    installment_pay[i] = installment_pay[i]/(-365.25)

installment_pay['PAID_OVER_AMOUNT'] = installment_pay['AMT_PAYMENT'] - installment_pay['AMT_INSTALMENT']    
installment_pay['PAID_OVER'] = (installment_pay['PAID_OVER_AMOUNT'] > 0).astype(int)
# Days past due and Days before due
installment_pay['DPD'] = installment_pay['DAYS_ENTRY_PAYMENT'] - installment_pay['DAYS_INSTALMENT']
installment_pay['DPD'] = installment_pay['DPD'].apply(lambda x: 0 if x <= 0 else x)
installment_pay['DBD'] = installment_pay['DAYS_INSTALMENT'] - installment_pay['DAYS_ENTRY_PAYMENT']
installment_pay['DBD'] = installment_pay['DBD'].apply(lambda x: 0 if x <= 0 else x)
# Flag late installment_payment
installment_pay['LATE_PAYMENT'] = installment_pay['DBD'].apply(lambda x: 1 if x > 0 else 0)
# Percentage of installment_payments that were late
installment_pay['INSTALMENT_PAYMENT_RATIO'] = installment_pay['AMT_PAYMENT'] / installment_pay['AMT_INSTALMENT']
installment_pay['LATE_PAYMENT_RATIO'] = installment_pay.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)

In [12]:
df = train_file.append(test_file)
del train_file,test_file
gc.collect()
print("test file merged")

test file merged


# Bureau and Bureau Balance


In [13]:
bb, categorical_cols, original_cols = one_hot_encoder(bureau_bal, nan_as_category= False)
bbp = bb.groupby('SK_ID_BUREAU')[categorical_cols+original_cols].mean().reset_index()

In [14]:
del bureau_bal,bb
gc.collect()

20

In [15]:
bureau, cat_cols, org_cols = one_hot_encoder(bureau, nan_as_category = False)
bp = bureau.merge(bbp, how='left', on='SK_ID_BUREAU')

In [16]:
del bureau
gc.collect()

40

In [17]:
bp = bp.groupby('SK_ID_BUREAU')
org_cols.remove("SK_ID_BUREAU")

bureau_p = pd.DataFrame()
bureau_p[cat_cols+org_cols] = bp[cat_cols+org_cols].first()
bureau_p = bureau_p.groupby('SK_ID_CURR').sum().reset_index()

In [18]:
del bbp,bp
gc.collect()

20

# POS Cash

In [19]:
pos, cat_cols, org_cols = one_hot_encoder(pos_cash)
posp = pos.groupby('SK_ID_PREV')
org_cols.remove("SK_ID_PREV")

In [20]:
pos_p = pd.DataFrame()
#pos_p[cat_cols+org_cols] = posp[cat_cols+org_cols].first()
pos_p = posp[cat_cols+org_cols].first().groupby('SK_ID_CURR').sum().reset_index()

In [21]:
del pos,posp,pos_cash
gc.collect()

60

# Installment Payments

In [22]:
inst, cat_cols, org_cols = one_hot_encoder(installment_pay)
instp = inst.groupby('SK_ID_PREV')
org_cols.remove('SK_ID_PREV')

In [23]:
inst_p = pd.DataFrame()
#inst_p[cat_cols+org_cols] = instp[cat_cols+org_cols].first()
inst_p = instp[cat_cols+org_cols].first().groupby('SK_ID_CURR').sum().reset_index()

In [24]:
del inst,instp,installment_pay
gc.collect()

60

# Previous Application

In [25]:
prev, cat_cols, org_cols = one_hot_encoder(prev_app)
prevp = prev.groupby('SK_ID_PREV')
org_cols.remove('SK_ID_PREV')

In [26]:
prev_p = pd.DataFrame()
#prev_p[cat_cols+org_cols] = prevp[cat_cols+org_cols].first()
prev_p = prevp[cat_cols+org_cols].first().groupby('SK_ID_CURR').sum().reset_index()

In [27]:
del prev,prevp,prev_app
gc.collect()

40

# Credit Card

In [28]:
credit_card, cat_cols, org_cols = one_hot_encoder(credit_card_bal)
credit_cardp = credit_card.groupby('SK_ID_PREV')
org_cols.remove('SK_ID_PREV')

In [29]:
credit_card_p = pd.DataFrame()
#credit_card_p[cat_cols+org_cols] = credit_cardp[cat_cols+org_cols].first()
credit_card_p = credit_cardp[cat_cols+org_cols].first().groupby('SK_ID_CURR').sum().reset_index()

In [30]:
del credit_card,credit_card_bal,credit_cardp
gc.collect()

60

In [31]:
df.shape

(307511, 133)

In [32]:
df = pd.merge(df,bureau_p, on='SK_ID_CURR', how='left')
del bureau_p
gc.collect()
#df = reduce_memory(df)
print("Bureau Merged")
print("Shape of df",df.shape)

df = pd.merge(df,prev_p, on='SK_ID_CURR', how='left')
del prev_p
gc.collect()
#df = reduce_memory(df)
print("Previous App merged")
print("Shape of df",df.shape)

df = pd.merge(df,pos_p, on='SK_ID_CURR', how='left')
del pos_p
gc.collect()
#df = reduce_memory(df)
print("pos_cash merged")
print("Shape of df",df.shape)

df = pd.merge(df,credit_card_p, on='SK_ID_CURR', how='left')
del credit_card_p
gc.collect()
#df = reduce_memory(df)
print("CC Bal merged")
print("Shape of df",df.shape)

df = pd.merge(df,inst_p, on='SK_ID_CURR', how='left')
del inst_p
gc.collect()
#df = reduce_memory(df)
print("Installments merged")
print("Shape of df",df.shape)

Bureau Merged
Shape of df (307511, 173)
Previous App merged
Shape of df (307511, 355)
pos_cash merged
Shape of df (307511, 371)
CC Bal merged
Shape of df (307511, 402)
Installments merged
Shape of df (307511, 415)


In [33]:
df = df.drop("SK_ID_CURR",axis=1)

In [34]:
DF,cat_cols,org_cols = one_hot_encoder(df)

In [35]:
DF.shape

(307511, 554)

In [36]:
DF_1 = DF
import re
DF_1 = DF_1.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [37]:
DF_1

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE_x,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Stonebrick,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,0.0,0,360000.0,1125000.0,33025.5,1125000.0,0.022800,51.863107,20.175222,6.442163,...,0,0,0,0,0,0,1,0,0,1
1,0.0,0,112500.0,251280.0,13630.5,180000.0,0.022625,38.039699,3.813826,15.600274,...,0,0,0,0,0,0,1,0,0,1
2,0.0,0,225000.0,544491.0,15916.5,454500.0,0.035792,54.266940,5.681040,5.806982,...,0,0,0,1,0,0,0,1,0,0
3,0.0,2,211500.0,900000.0,26316.0,900000.0,0.006207,35.685147,6.036961,3.266256,...,0,0,0,0,0,0,1,0,0,1
4,1.0,0,90000.0,113760.0,8406.0,90000.0,0.020713,50.357290,4.637919,22.666667,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,NaN,1,202500.0,835380.0,40320.0,675000.0,0.035792,35.800137,5.081451,18.880219,...,0,0,0,0,0,0,1,0,0,1
307507,NaN,0,450000.0,1800000.0,56520.0,1800000.0,0.002506,40.002738,20.101300,23.671458,...,0,0,0,0,0,0,0,1,0,0
307508,NaN,1,112500.0,301095.0,23773.5,279000.0,0.019101,42.737851,5.932923,0.325804,...,0,0,0,0,0,0,1,0,0,1
307509,NaN,0,94500.0,180000.0,9000.0,180000.0,0.011703,22.450376,1.686516,22.439425,...,0,0,0,0,1,0,0,1,0,0


In [38]:
DF_1.to_csv('df.csv',index=False)